In [1]:
# Python 3.10

import pandas as pd
import keras
import keras_nlp
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

Using TensorFlow backend


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

c:\Users\gianmariafredacivico\.conda\envs\Kaggle\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Creo gli embedding multi lingue

In [3]:
embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/multilingual/versions/2")

In [4]:
eng_df = pd.read_csv('data/eng_df.csv', low_memory=False, index_col=[0])
jap_df = pd.read_csv('data/jap_df.csv', low_memory=False, index_col=[0])

eng_df = eng_df.drop_duplicates()
jap_df = jap_df.drop_duplicates()

eng_df = eng_df[eng_df.words.apply(len) > 0]
jap_df = jap_df[jap_df.words.apply(len) > 0]

eng_df = eng_df.dropna(subset='text')
jap_df = jap_df.dropna(subset='text')

In [ ]:
df_full = pd.concat([eng_df, jap_df])

df_full['embedded_text'] = df_full.text.progress_apply(lambda x: embed(x))
df_full.to_csv('data/dataset_full_embedding.csv')

100%|██████████| 348935/348935 [2:39:12<00:00, 36.53it/s]  


# Modelli su embedding multilingue

In [5]:
from tqdm import tqdm 
tqdm.pandas()

In [6]:
df_full = pd.read_csv('data/dataset_full_embedding.csv', index_col=[0])

C:\Users\gianmariafredacivico\AppData\Local\Temp\ipykernel_9600\2121335445.py:1: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv('data/dataset_full_embedding.csv', index_col=[0])


In [7]:
X = df_full['embedded_text'].progress_apply(lambda x: [float(num) for num in x.split("[[")[1].split("]]")[0].split()]).values
y = df_full['source'].values

100%|██████████| 348935/348935 [01:02<00:00, 5607.23it/s]


# Smote esegue il resampling bilanciando le classi

In [8]:
smote = SMOTE(random_state=42)

X_resampled, y_resampled = smote.fit_resample(np.array([i for i in X]), y)

In [9]:
import joblib 


joblib.dump(smote, 'models/preprocessing/multi/smote.pkl')

['models/preprocessing/multi/smote.pkl']

In [7]:
unique, counts = np.unique(y_resampled, return_counts=True)
class_counts = dict(zip(unique, counts))
print("Numero di campioni per classe dopo l'applicazione di SMOTE:")
print(class_counts)

Numero di campioni per classe dopo l'applicazione di SMOTE:
{'Mainichi Shimbun': 44656, 'The Japan Times': 44656, 'asahi.com': 44656, 'chunichi.co.jp': 44656, 'hokkaido-np.co.jp': 44656, 'isenp.co.jp': 44656, 'iwate-np.co.jp': 44656, 'kobe-np.co.jp': 44656, 'kyoto-np.co.jp': 44656, 'mainichi.jp': 44656, 'nikkansports.com': 44656, 'nikkei.com': 44656, 'nishinippon.co.jp': 44656, 'nnn.co.jp': 44656, 'oita-press.co.jp': 44656, 'sankei.jp.msn.com': 44656, 'sanspo.com': 44656, 'shimotsuke.co.jp': 44656, 'tokachi.co.jp': 44656, 'tokyo-np.co.jp': 44656, 'tomamin.co.jp': 44656, 'yamagata-np.jp': 44656, 'yomiuri.co.jp': 44656}


In [8]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [9]:
import os

def mkdir(folder_path):

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Cartella '{folder_path}' creata con successo.")
    else:
        print(f"La cartella '{folder_path}' esiste già.")

In [10]:
mkdir('models/preprocessing/multi')

joblib.dump(label_encoder, 'models/preprocessing/multi/label_encoder.pkl')

La cartella 'models/preprocessing/multi' esiste già.


['models/preprocessing/multi/label_encoder.pkl']

In [11]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(821670, 512) (821670,) (205418, 512) (205418,)


In [12]:
def plot_confusion_matrix(y_true, y_pred, path):
    cm = confusion_matrix(y_true, y_pred)
    labels = sorted(set(y_true))
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.savefig(path)
    plt.close()

def evaluate_model(y_true, y_pred, model_name, metrics_path):
    report = classification_report(y_true, y_pred, output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    accuracy = accuracy_score(y_true, y_pred)
    
    with open(metrics_path, 'a') as file:
        file.write(f"Metrics for {model_name}:\n\n")
        file.write(f"Accuracy: {accuracy:.4f}\n\n")
        file.write("Classification Report:\n")
        file.write(df_report.to_string())
        file.write("\n\n")

# Provo due modelli (troppo tempo)
models = {
    # "random_forest": RandomForestClassifier(),
    "xgboost": XGBClassifier(),
}

mkdir('models/multi/')

print('starting')

for model_name, model in models.items():
    print(f"Fitting {model_name}...")


    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    plot_confusion_matrix(y_test, y_pred, f"models/multi/{model_name}_confusion_matrix.png")
    
    evaluate_model(y_test, y_pred, model_name, f"models/multi/models_metrics.txt")

La cartella 'models/multi/' esiste già.
starting
Fitting xgboost...


c:\Users\gianmariafredacivico\.conda\envs\Kaggle\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:28:11] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
